Exercício 1

In [ ]:
import numpy as np
from scipy.linalg import solve_continuous_lyapunov

A = np.array([[0, 1],
              [-2, -3]])
Q = np.eye(2) 

# A'P + PA + Q = 0
P = solve_continuous_lyapunov(A.T, Q)

print("Matriz P:")
print(P)
print("\nTraço de P (Tr(P)):", np.trace(P))

In [ ]:
import cvxpy as cp
import numpy as np

A = np.array([[0, 1],
              [-2, -3]])
Q = np.eye(2) 

P = cp.Variable((2, 2), symmetric=True)

constraints = [
    P >> 0,  # P > 0 (definida positiva)
    A.T @ P + P @ A + Q << 0  # A'P + PA + Q < 0 (definida negativa)
]

objective = cp.Minimize(cp.trace(P))

problem = cp.Problem(objective, constraints)

problem.solve()

print("Matriz P (solução ótima):")
print(P.value)
print("\nTraço de P (Tr(P)):", np.trace(P.value))

Exercício 2

In [ ]:
import cvxpy as cp
import numpy as np

# System parameters
m1 = 1
m2 = 0.5
k1 = 1
k2 = 1
c0 = 2

A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-((k1 + k2) / m1), k2 / m1, -c0 / m1, 0],
    [k2 / m2, -k2 / m2, 0, -c0 / m2]
])

B = np.array([
    [0],
    [0],
    [1 / m1],
    [0]
])

C = np.array([[0, 1, 0, 0]])

n_states = A.shape[0]

W = cp.Variable((n_states, n_states), symmetric=True)
rho = cp.Variable()

objective = cp.Minimize(rho)

constraints = [
    W >> 0, 
    rho >= cp.trace(C @ W @ C.T),
    A @ W + W @ A.T + B @ B.T << 0
]

problem = cp.Problem(objective, constraints)
problem.solve()

if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    rho_star = rho.value
    h2_norm_squared = rho_star
    
    print(f'Status da solução: {problem.status}')
    print(f'Valor ótimo de \u03C1 (norma H2 ao quadrado): {h2_norm_squared:.4f}')
    print(f'Norma H2: {np.sqrt(h2_norm_squared):.4f}')

    print('Matriz W ótima:')
    print(W.value)
else:
    print(f'O problema não foi resolvido com sucesso. Status: {problem.status}')

Status da solução: optimal
Valor ótimo de ρ (norma H2 ao quadrado): 0.0926
Norma H2: 0.3043
Matriz W ótima:
[[ 1.57407421e-01  8.33333105e-02 -2.78350935e-08  3.70370627e-02]
 [ 8.33333105e-02  9.25925906e-02 -3.70370932e-02 -1.61823818e-08]
 [-2.78350935e-08 -3.70370932e-02  2.31481473e-01 -4.69721454e-09]
 [ 3.70370627e-02 -1.61823818e-08 -4.69721454e-09  1.85185400e-02]]


Exercício 3

In [ ]:
import cvxpy as cp
import numpy as np

m1 = 1
m2 = 0.5
k1 = 1
k2 = 1
c0 = 2

A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-((k1 + k2) / m1), k2 / m1, -c0 / m1, 0],
    [k2 / m2, -k2 / m2, 0, -c0 / m2]
])

B = np.array([
    [0],
    [0],
    [1 / m1],
    [0]
])

C = np.array([[0, 1, 0, 0]])

D = np.zeros((1, 1))  # Zeros

n_states = A.shape[0]  # States number(4)
n_outputs = C.shape[0]  # Outputs number (1)

P = cp.Variable((n_states, n_states), symmetric=True)  
mu = cp.Variable() 

objective = cp.Minimize(mu)

block11 = A.T @ P + P @ A
block12 = P @ B
block13 = C.T
block21 = B.T @ P
block22 = -mu * np.eye(1)  
block23 = np.zeros((1, 1)) 
block31 = C
block32 = np.zeros((1, 1)) 
block33 = -np.eye(1) 

lmi_matrix = cp.bmat([
    [block11, block12, block13],
    [block21, block22, block23],
    [block31, block32, block33]
])

constraints = [
    P >> 0,
    lmi_matrix << 0 
]

problem = cp.Problem(objective, constraints)
problem.solve()

if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    mu_star = mu.value
    h_inf_norm_squared = mu_star
    h_inf_norm = np.sqrt(mu_star)

    print(f'Status: {problem.status}')
    print(f'Valor ótimo (norma H infinito ao quadrado): {h_inf_norm_squared:.4f}')
    print(f'Norma H infinito: {h_inf_norm:.4f}')
    print('Matriz P ótima:')
    print(P.value)
else:
    print(f'Problema não resolvido. Status: {problem.status}')

Status da solução: optimal
Valor ótimo de μ (norma H∞ ao quadrado): 1.0000
Norma H∞: 1.0000


Exercício 4

In [12]:
import numpy as np
import cvxpy as cp
from itertools import product

# 1) Dimensões do sistema (preencha conforme seu modelo do Exercício 4)
n_x = 4  # estados
n_u = 1  # entrada de controle (T)
n_w = 1  # entrada de perturbação (w)
n_y = 3  # saída de desempenho (z ou y que define o custo H2)

print(f"n_x={n_x}, n_u={n_u}, n_w={n_w}, n_y={n_y}")

# 2) Matrizes constantes B1, B2, C, D (PRENCHA AQUI)
# Formatos: B1 (n_x x n_w), B2 (n_x x n_u), C (n_y x n_x), D (n_y x n_u)
# Substitua os TODOs pelos valores do seu Exercício 4.
B1_const = np.array([
    # TODO: exemplo para n_w=1
    [0.0],
    [0.0],
    [0.0],
    [1.0],
], dtype=float)

B2_const = np.array([
    # TODO: exemplo para n_u=1
    [0.0],
    [0.0],
    [0.0],
    [1.0],
], dtype=float)

C_const = np.array([
    # TODO: 3 x 4
    # Exemplo: medir dois ângulos e uma combinação de velocidades
    [1.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 0.1, 0.1],
], dtype=float)

D_const = np.array([
    # TODO: 3 x 1 (feedthrough do controle para a saída de desempenho)
    [0.0],
    [0.0],
    [0.01],
], dtype=float)

# 3) Faixas de incerteza (Exercício 6)
k_min, k_max = 0.09, 0.4
f_min, f_max = 0.0038, 0.04

# 4) A(k,f) para os vértices do politopo
# Defina A(k, f) de modo afim (PRENCHA AQUI com seu modelo)
def A_of(k, f):
    # Exemplo ilustrativo (substitua pelo seu A do Exercício 4):
    A0  = np.array([[0, 0, 1, 0],
                    [0, 0, 0, 1],
                    [0, 0, 0, 0],
                    [0, 0, 0, 0]], dtype=float)
    A_k = np.array([[0, 0, 0, 0],
                    [0, 0, 0, 0],
                    [-1, 1, 0, 0],
                    [ 1,-1, 0, 0]], dtype=float)
    A_f = np.array([[0, 0, 0, 0],
                    [0, 0, 0, 0],
                    [0, 0,-1, 1],
                    [0, 0, 1,-1]], dtype=float)
    return A0 + k*A_k + f*A_f

# Construa os vértices (k,f) nos extremos
vertices = []
for k_val, f_val in product([k_min, k_max], [f_min, f_max]):
    Ai = A_of(k_val, f_val)
    vertices.append((Ai, B1_const, B2_const, C_const, D_const))

print(f"Número de vértices do politopo: {len(vertices)}")

# 5) Variáveis de decisão
W   = cp.Variable((n_x, n_x), symmetric=True)     # W = P^{-1}
Z   = cp.Variable((n_u, n_x))                     # Z = K W
X   = cp.Variable((n_y, n_y), symmetric=True)     # para custo H2 (forma com C,D)
rho = cp.Variable()                               

# 6) Restrições (LMIs) — forma padrão H2 (consistente)
constraints = []
eps = 1e-8
constraints.append(W >> eps*np.eye(n_x))
constraints.append(X >> eps*np.eye(n_y))
constraints.append(rho >= cp.trace(X))

for (Ai, B1i, B2i, Ci, Di) in vertices:
    # LMI de desempenho H2:
    # [ X, Ci W + Di Z ; (Ci W + Di Z)' , W ] > 0
    CW_DZ = Ci @ W + Di @ Z
    constraints.append(cp.bmat([[X,          CW_DZ],
                                [CW_DZ.T,     W   ]]) >> eps*np.eye(n_y + n_x))

    # LMI de estabilidade com perturbação:
    # [ Ai W + B2i Z + (Ai W + B2i Z)' , B1i ; B1i' , -I ] < 0
    AclW = Ai @ W + B2i @ Z
    M11  = AclW + AclW.T
    constraints.append(cp.bmat([[M11,       B1i],
                                [B1i.T, -np.eye(n_w)]]) << -eps*np.eye(n_x + n_w))

# 7) Problema e solução
problem = cp.Problem(cp.Minimize(rho), constraints)
# Escolha do solver (SCS é open-source; MOSEK é mais robusto se disponível)
problem.solve(solver=cp.SCS, verbose=False)

print("\nStatus:", problem.status)
if problem.status in ("optimal", "optimal_inaccurate"):
    print("rho* (limite garantido de ||H||_2^2):", float(rho.value))
    W_val = W.value
    Z_val = Z.value
    try:
        K = Z_val @ np.linalg.inv(W_val)
        print("K (ganho de realimentação de estados):", K)
    except np.linalg.LinAlgError:
        print("W não invertível. Tente outro solver, ajustar eps ou reescalar o problema.")
else:
    print("Problema inviável/não ótimo. Verifique dimensões e matrizes.")

n_x=4, n_u=1, n_w=1, n_y=3
Número de vértices do politopo: 4

Status: optimal_inaccurate
rho* (limite garantido de ||H||_2^2): 0.0008535386513020679
K (ganho de realimentação de estados): [[  26.83511421 -119.85651611 -374.95968163  -19.77702044]]


Exercício 5

In [ ]:
import cvxpy as cp
import numpy as np

n = 4  # Dimensão do estado (theta1, theta2, dtheta1, dtheta2)
mu = 1 # Dimensão da entrada de controle (T)
mw = 1 # Dimensão da perturbação (w)
py = 3 # Dimensão da saída (y)

k = 1   # Constante da mola (valor nominal)
f = 1  # Amortecimento viscoso (valor nominal)

A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-k, k, -f, f],
    [k, -k, f, -f]
])

B1 = np.array([
    [0],
    [0],
    [0],
    [1]
])

B2 = np.array([
    [0],
    [0],
    [0],
    [1]
])

C = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0]
])

D = np.array([
    [0],
    [0],
    [1]
])

# mu: escalar a ser minimizado (quadrado da norma H_infinito)
m_u = cp.Variable()

Z = cp.Variable((mu, n))

W = cp.Variable((n, n), symmetric=True)

objective = cp.Minimize(mu)

constraints = [
    # LMI to H_inf gain
    cp.bmat([
        [A @ W + W @ A.T + B2 @ Z + Z.T @ B2.T, W @ C.T + Z.T @ D.T, B1],
        [C @ W + D @ Z, -np.eye(py), np.zeros((py, mw))],
        [B1.T, np.zeros((mw, py)), -m_u * np.eye(mw)]
    ]) << 0,  # Defined Negative Matrix

    # W positive defined
    W >> 0,

   # mu is not negative
    m_u >= 0
]

problem = cp.Problem(objective, constraints)
problem.solve(solver='SCS')

# Results
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    print('--- Solução Ótima Encontrada ---')
    print(f'Status do problema: {problem.status}')

    # Value of the cost H_in squared
    h_inf_norm_squared = m_u.value
    print(f'Custo H_infinito ao quadrado (µ*): {h_inf_norm_squared:.4f}')
    
    # H_inf Norm 
    if h_inf_norm_squared >= 0:
        h_inf_norm = np.sqrt(h_inf_norm_squared)
        print(f'Norma H_infinito (√µ*): {h_inf_norm:.4f}')
    else:
        print('Erro: O valor de mu é negativo, a norma H_infinito não pode ser calculada.')

    K = Z.value @ np.linalg.inv(W.value)
    print('Matriz de ganho de realimentação de estados K:')
    print(K)

    A_cl = A + B2 @ K
    eigvals_cl = np.linalg.eigvals(A_cl)
    print('Autovalores do sistema em malha fechada (parte real):')
    print(np.real(eigvals_cl))
    if np.max(np.real(eigvals_cl)) < 0:
        print('O sistema em malha fechada é assintoticamente estável.')
    else:
        print('O sistema em malha fechada NÃO é assintoticamente estável.')

else:
    print(f'\nO problema não convergiu para uma solução ótima. Status: {problem.status}')

--- Solução Ótima Encontrada ---
Status do problema: optimal
Custo H_infinito ao quadrado (µ*): 1.7903
Norma H_infinito (√µ*): 1.3380
Matriz de ganho de realimentação de estados K:
[[ -3.78036209  -5.09957617  -8.69540831 -10.72031841]]

Autovalores do sistema em malha fechada (parte real):
[-10.43047719  -0.90252211  -0.90252211  -0.48479701]
O sistema em malha fechada é assintoticamente estável.


Exercício 6

In [13]:
import numpy as np
import cvxpy as cp
from itertools import product

# Dimensões (ajuste conforme seu caso)
n = 4      # estados
m = 1      # entradas de controle u
n_w = 1    # entradas de perturbação w
p_out = 3  # saídas de desempenho z (para custo H2)

# Intervalos de incerteza
k_min, k_max = 0.09, 0.4
f_min, f_max = 0.0038, 0.04

# Modelo afim ilustrativo (substitua pelo do exercício)
def A_of(k, f):
    A0  = np.array([[0, 0, 1, 0],
                    [0, 0, 0, 1],
                    [0, 0, 0, 0],
                    [0, 0, 0, 0]], float)
    A_k = np.array([[0, 0, 0, 0],
                    [0, 0, 0, 0],
                    [-1, 1, 0, 0],
                    [ 1,-1, 0, 0]], float)
    A_f = np.array([[0, 0, 0, 0],
                    [0, 0, 0, 0],
                    [0, 0,-1, 1],
                    [0, 0, 1,-1]], float)
    return A0 + k*A_k + f*A_f

# B1 (n x n_w), B2 (n x m), C (p_out x n), D (p_out x m) — substitua pelos seus
B1 = np.array([[0.0],
               [0.0],
               [0.0],
               [1.0]], float)

B2 = np.array([[0.0],
               [0.0],
               [0.0],
               [1.0]], float)

C = np.array([[1.0, 0.0, 0.0, 0.0],
              [0.0, 1.0, 0.0, 0.0],
              [0.0, 0.0, 0.1, 0.1]], float)

D = np.array([[0.0],
              [0.0],
              [0.01]], float)

# Vértices politópicos
vertices = [(A_of(k, f), B1, B2, C, D)
            for k, f in product([k_min, k_max], [f_min, f_max])]

# Variáveis de decisão
W   = cp.Variable((n, n), symmetric=True)
Z   = cp.Variable((m, n))
X   = cp.Variable((p_out, p_out), symmetric=True)
rho = cp.Variable()

constraints = []
eps = 1e-8

# W > 0
constraints.append(W >> eps*np.eye(n))

# LMIs em todos os vértices
for (Ai, B1i, B2i, Ci, Di) in vertices:
    # Desempenho H2
    CW_DZ = Ci @ W + Di @ Z
    constraints.append(cp.bmat([[X,          CW_DZ],
                                [CW_DZ.T,     W   ]]) >> eps*np.eye(p_out + n))

    # Estabilidade + w
    AclW = Ai @ W + B2i @ Z
    M11  = AclW + AclW.T
    constraints.append(cp.bmat([[M11,       B1i],
                                [B1i.T, -np.eye(n_w)]]) << -eps*np.eye(n + n_w))

# Ligação do objetivo
constraints.append(rho >= cp.trace(X))

# Objetivo
prob = cp.Problem(cp.Minimize(rho), constraints)
prob.solve(solver=cp.SCS, verbose=False)  # ou MOSEK, se disponível

print("Status:", prob.status)
if prob.status in ("optimal", "optimal_inaccurate"):
    print("rho*:", float(rho.value))
    K = Z.value @ np.linalg.inv(W.value)
    print("K:", K)
else:
    print("Problema inviável/não ótimo. Verifique matrizes e dimensões.")

Status: optimal_inaccurate
rho*: 0.0008503601403516594
K: [[  61.49479163 -118.19139177 -387.54742608  -19.75836606]]


Exercício 7

In [ ]:
import numpy as np
import cvxpy as cp

M0 = np.array([
    [-2.4, -0.6, -1.7,  3.1],
    [ 0.7, -2.1, -2.6, -3.6],
    [ 0.5,  2.4, -5.0, -1.6],
    [-0.6,  2.9, -2.0, -0.6]
], dtype=float)

M1 = np.array([
    [ 1.1, -0.6, -0.3, -0.1],
    [-0.8,  0.2, -1.1,  2.8],
    [-1.9,  0.8, -1.1,  2.0],
    [-2.4, -3.1, -3.7, -0.1]
], dtype=float)

M2 = np.array([
    [ 0.9,  3.4,  1.7,  1.5],
    [-3.4, -1.4,  1.3,  1.4],
    [ 1.1,  2.0, -1.5, -3.4],
    [-0.4,  0.5,  2.3,  1.5]
], dtype=float)

M3 = np.array([
    [-1.0, -1.4, -0.7, -0.7],
    [ 2.1,  0.6, -0.1, -2.1],
    [ 0.4, -1.4,  1.3,  0.7],
    [ 1.5,  0.9,  0.4, -0.5]
], dtype=float)

M_list = [M1, M2, M3]

n = M0.shape[0]
N = len(M_list)

# =========================
# Parâmetros numéricos
# =========================
eps = 1e-8         # margem para <0 e >0
rho_lo = 0.0       # limite inferior inicial da bisseção
rho_hi = 100.0      # limite superior inicial (ajuste se necessário)
max_iter = 50      # iterações de bisseção
verbose_feas = False  # ativar logs do solver por iteração, se quiser
solver_choice = cp.SCS  # ou cp.MOSEK, se disponível

# =========================
# Checagem de factibilidade
# =========================
def is_feasible(rho_val):
    """
    Testa factibilidade das LMIs para um rho fixo.
    Retorna (viavel: bool, P_solutions: list of np.ndarray or None).
    """
    # Constrói os A_j = M0 + rho * Mj como constantes
    A_list = [M0 + rho_val * M_list[j] for j in range(N)]

    # Variáveis de Lyapunov por vértice (P_j)
    P = [cp.Variable((n, n), symmetric=True) for _ in range(N)]
    cons = []

    # P_j >> 0
    for Pj in P:
        cons.append(Pj >> eps * np.eye(n))

    # A_j' P_j + P_j A_j << 0
    for j in range(N):
        Aj = A_list[j]
        cons.append(Aj.T @ P[j] + P[j] @ Aj << -eps * np.eye(n))

    # A_i' P_j + P_j A_i + A_j' P_i + P_i A_j << 0, para i < j
    for i in range(N):
        for j in range(i + 1, N):
            Ai = A_list[i]
            Aj = A_list[j]
            Pi = P[i]
            Pj = P[j]
            cons.append(Ai.T @ Pj + Pj @ Ai + Aj.T @ Pi + Pi @ Aj << -eps * np.eye(n))

    prob = cp.Problem(cp.Minimize(0), cons)
    try:
        prob.solve(solver=solver_choice, verbose=verbose_feas)
        feasible = prob.status in ("optimal", "optimal_inaccurate")
        if feasible:
            return True, [Pj.value for Pj in P]
        else:
            return False, None
    except cp.error.SolverError:
        return False, None

best_rho = rho_lo
best_Ps = None
# find better rho
for k in range(max_iter):
    rho_mid = 0.5 * (rho_lo + rho_hi)
    feasible, P_vals = is_feasible(rho_mid)
    if feasible:
        best_rho = rho_mid
        best_Ps = P_vals
        rho_lo = rho_mid  # tenta aumentar rho
    else:
        rho_hi = rho_mid  # diminui rho

print("Status bisseção: concluída")
print(f"rho* (aprox.): {best_rho:.6f}")
if best_Ps is not None:
    for j, Pj in enumerate(best_Ps, 1):
        print(f"\nP_{j} (aprox. no rho*):\n{Pj}")
else:
    print("Não foi possível recuperar P_j. Tente ajustar rho_hi, max_iter, eps, ou usar MOSEK.")

Status bisseção: concluída
rho* (aprox.): 100.000000

P_1 (aprox. no rho*):
[[ 1.35365643e-08  3.08606936e-09  1.70408841e-09 -2.29974216e-09]
 [ 3.08606936e-09  1.33198041e-08  4.26791448e-09 -1.86342873e-09]
 [ 1.70408841e-09  4.26791448e-09  7.87663339e-09 -1.19220319e-09]
 [-2.29974216e-09 -1.86342873e-09 -1.19220319e-09  1.23841857e-08]]

P_2 (aprox. no rho*):
[[ 1.27059968e-08  2.63098978e-09  1.70904821e-09 -2.22256763e-09]
 [ 2.63098978e-09  1.35230084e-08  4.44394456e-09 -1.43393404e-09]
 [ 1.70904821e-09  4.44394456e-09  9.17875567e-09 -1.23387380e-09]
 [-2.22256763e-09 -1.43393404e-09 -1.23387380e-09  1.30845935e-08]]

P_3 (aprox. no rho*):
[[ 1.16122790e-08  2.69054095e-09  1.86008140e-09 -2.79128179e-09]
 [ 2.69054095e-09  1.28228914e-08  4.15777483e-09 -1.64182559e-09]
 [ 1.86008140e-09  4.15777483e-09  8.34745565e-09 -1.43622935e-09]
 [-2.79128179e-09 -1.64182559e-09 -1.43622935e-09  1.13867728e-08]]


Exercício 8

In [ ]:
import numpy as np
import cvxpy as cp

A1 = np.array([[ 5.3,  5.0],
               [-5.3, -12.7]], float)
A2 = np.array([[-13.0, 20.9],
               [ -0.6, -14.2]], float)
B1 = np.array([[-0.2],
               [-1.2]], float)  # se for a entrada de controle, ajuste conforme sua referência
B2 = np.array([[0.0],
               [0.1]], float)

A_list = [A1, A2]
B_list = [B1, B2]   # verifique se é essa B que entra na LMI
N = len(A_list)

n = A1.shape[0]
m = B1.shape[1]

eps = 1e-8
xi_values = np.linspace(0.1, 5.0, 50)

best = None

for xi in xi_values:
    W = cp.Variable((n, n), symmetric=True)
    Z = cp.Variable((m, n))

    cons = [ W >> eps*np.eye(n) ]

    for Ai, Bi in zip(A_list, B_list):
        # EXEMPLO DE BLOCO LMI LINEAR EM (W,Z) — SUBSTITUA PELA SUA LMI
        AW_BZ = Ai @ W + Bi @ Z
        # Aqui coloco um exemplo de matriz em bloco simétrica típica:
        M = cp.bmat([
            [AW_BZ + AW_BZ.T,       W],
            [      W,         -xi*np.eye(n)]
        ])
        cons.append(M << -eps*np.eye(2*n))

    obj = cp.Minimize(cp.norm(W, 'fro') + cp.norm(Z, 'fro'))

    prob = cp.Problem(obj, cons)
    try:
        prob.solve(solver=cp.SCS, verbose=False)
    except cp.error.SolverError:
        continue

    if prob.status in (cp.OPTIMAL, cp.OPTIMAL_INACCURATE):
        try:
            K = Z.value @ np.linalg.inv(W.value)
            best = (xi, K, W.value, Z.value, prob.status)
            print(f"Factível em xi={xi:.3f}.")
            break
        except np.linalg.LinAlgError:
            continue

if best is not None:
    xi_star, K_star, W_star, Z_star, status = best
    print(f"\nStatus: {status}")
    print(f"xi*: {xi_star:.4f}")
    print("K:\n", K_star)
else:
    print("\nNenhuma solução factível com a LMI de exemplo. Ajuste a LMI para a sua referência.")

Factível em xi=0.100.

Status: optimal
xi*: 0.1000
K:
 [[0.85217229 0.07923362]]
